<a href="https://colab.research.google.com/github/arthurbabey/road66/blob/master/cnn_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [3]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import sys
sys.path.append('/Users/arthurbabey/Documents/master2/ML/road66/scripts')
import keras 


from skimage.transform import rotate
from sklearn.model_selection import train_test_split


from keras import regularizers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg




# Convert array of labels to an image

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im

def value_to_class(v, foreground_threshold=0.25): #modifier leur fonction en ajoutant f_t en param

    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def rotate_images(X, Y , degrees):
    """
    increase the number of data
    by adding rotations of the base data
    """

    X = np.array(X)
    Y = np.array(Y)
    rotimg = np.zeros(X.shape)
    rotgtimg = np.zeros(Y.shape)
    
    Xtemp = X
    Ytemp = Y
    
    #rotate all images by degree and add them to the data vector
    for degree in degrees:
        for i in range(len(Xtemp)):
            rotimg[i] = rotate(Xtemp[i], degree, resize=False, mode='reflect')
            rotgtimg[i] = rotate(Ytemp[i], degree, resize=False, mode='reflect')
        X = np.concatenate([X,rotimg])
        Y = np.concatenate([Y,rotgtimg])

    return X,Y


def create_submission(y_pred, submission_filename, patch_size = 16, images_size = 608):
    n_patches = images_size // patch_size
    y_pred = np.reshape(y_pred, (-1, n_patches, n_patches))

    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for i in range(y_pred.shape[0]):
            img = y_pred[i]
            for j in range(img.shape[0]):
                for k in range(img.shape[1]):
                    name = '{:03d}_{}_{},{}'.format(i + 1, j * patch_size, k * patch_size, int(img[j,k]))
                    f.write(name + '\n')

def balance_data(x_train, y_train):
    c0 = 0  # bgrd
    c1 = 0  # road
    for i in range(len(y_train)):
        if y_train[i][0] == 1:
            c0 = c0 + 1
        else:
            c1 = c1 + 1
    print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

    print('Balancing training data...')
    min_c = min(c0, c1)
    idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
    idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
    new_indices = idx0[0:min_c] + idx1[0:min_c]
    print(len(new_indices))
    print(x_train.shape)
    x_train = x_train[new_indices, :, :, :]
    y_train = y_train[new_indices]

    train_size = y_train.shape[0]

    c0 = 0
    c1 = 0
    for i in range(len(y_train)):
        if y_train[i][0] == 1:
            c0 = c0 + 1
        else:
            c1 = c1 + 1
    print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

    return x_train, y_train 


def img_crop(im, w, h, border = 0, step = 16):
    """
    Return the patches list of an image.
    """
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    if border != 0:
        im_r = np.pad(im[:,:,0], ((border, border), (border, border)), 'reflect')
        im_g = np.pad(im[:,:,1], ((border, border), (border, border)), 'reflect')
        im_b = np.pad(im[:,:,2], ((border, border), (border, border)), 'reflect')
        im = np.dstack((im_r, im_g, im_b))
    for i in range(0,imgheight,step):
        for j in range(0,imgwidth,step):
            if is_2d:
                im_patch = im[j:j+w+2*border, i:i+h+2*border]
            else:
                im_patch = im[j:j+w+2*border, i:i+h+2*border, :]
            list_patches.append(im_patch)
    return list_patches

In [5]:
# Loaded a set of images
root_dir = "drive/My Drive/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) # Use all images
print("Loading " + str(n) + " testing images")
imgs = [load_image(image_dir + files[i]) for i in range(100)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(100)]

Loading 100 testing images
Loading 100 groundtruth


In [0]:
imgs, gt_imgs = rotate_images(imgs, gt_imgs, [15, 45, 60])

In [8]:
# 
# Preparation training data
# 
patch_size = 16 # each patch is 16*16 pixels
border = 16
n = len(imgs)

# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels


#img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
#gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

img_patches = [img_crop(imgs[i], patch_size, patch_size, border, step = 16) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size, step = 16) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
print(np.asarray(img_patches).shape)

#img_patches = standardize(img_patches)

img_patches = np.asarray(img_patches)
gt_patches = np.asarray(gt_patches)

x_train = img_patches
y_train = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(gt_patches.shape[0])])

print("62500 patches of 16x16 pixels for training are stored in x_train")
print("62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train")

(250000, 48, 48, 3)
62500 patches of 16x16 pixels for training are stored in x_train
62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train


In [0]:
del img_patches, gt_patches

In [0]:
split_data = True 

#split the x_train and y_train if True

if split_data:

    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                      test_size=0.13,  random_state=42)


In [0]:
im_train, im_test, gt_train, gt_test = train_test_split(img_patches, gt_patches, 
                                                        test_size=0.23, random_state= 42)


In [11]:
# 
# Preparation testing data to make a submission
# 

# 
# Ordering image
# 
root_testdir = "drive/My Drive/test_set_images"
test_names = os.listdir(root_testdir)


num_test = len(test_names)
order = [int(test_names[i].split("_")[1]) for i in range(num_test)]
index = np.argsort(order)

# Load image and reorder them
imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png") 
             for i in range(num_test)]
imgs_test = [imgs_test[i] for i in index]

# Crop images in patch and Linearize list of patches
patch_size = 16 # each patch is 16*16 pixels

#img_patches_test = [img_crop(imgs_test[i], patch_size, patch_size) for i in range(num_test)]

img_patches_test = [img_crop(imgs_test[i], patch_size, patch_size, border, step = 16) for i in range(num_test)]



# Linearize list of patches
x_testset = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) 
                     for j in range(len(img_patches_test[i]))])

#x_testset = standardize(x_testset)

print("72200 patches of 16x16 pixels from the test set are stored in y_test")

72200 patches of 16x16 pixels from the test set are stored in y_test


In [0]:
del img_patches_test, imgs_test

In [0]:
y_train = np.array(y_train, ndmin = 2)            
y_train = keras.utils.to_categorical(y_train.T, 2)

#y_val = np.array(y_val, ndmin = 2)            
#y_val = keras.utils.to_categorical(y_val.T, 2)

In [15]:
x_train, y_train = balance_data(x_train, y_train)

Number of data points per class: c0 = 186756 c1 = 63244
Balancing training data...
126488
(250000, 48, 48, 3)
Number of data points per class: c0 = 63244 c1 = 63244


In [0]:
batch_size = 32
num_classes = 2
epochs = 30


In [0]:
model = tf.keras.models.Sequential() #create a model

model.add(tf.keras.layers.Conv2D(32, (5, 5), activation = tf.nn.relu , padding='valid', #first conv + relu layer
                 data_format='channels_last',
                 input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) #first pooling layer
model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Conv2D(64, (5, 5), activation = tf.nn.relu )) #conv + relu layer
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) #pooling layer
model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Conv2D(128, (5, 5), activation = tf.nn.relu )) #conv + relu layer
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2))) #pooling layer
model.add(tf.keras.layers.Dropout(0.25))


model.add(tf.keras.layers.Flatten()) #flatt layer before feeding the fully connected layer

model.add(tf.keras.layers.Dense(256, activation = tf.nn.relu, use_bias=True, 
                                kernel_regularizer=keras.regularizers.l2(0.01), 
                                bias_regularizer=keras.regularizers.l2(0.01), 
                               activity_regularizer=keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax, use_bias=True, 
                                kernel_regularizer=keras.regularizers.l2(0.01), 
                                bias_regularizer=keras.regularizers.l2(0.01), 
                                activity_regularizer=keras.regularizers.l2(0.01)
))



In [25]:
model.compile(loss='binary_crossentropy', #compile the model
              optimizer='adam',
              metrics=['binary_accuracy'])


model.fit(x_train, y_train,              #fitting the model 
              batch_size=batch_size,
              epochs=epochs,
              #validation_data=(x_val, y_val),
              #callbacks = [tensorboard_callback]
         )

Train on 126488 samples
Epoch 1/200
126488/126488 [==============================] - 13s 101us/sample - loss: 0.5518 - binary_accuracy: 0.8089
Epoch 2/200
126488/126488 [==============================] - 12s 93us/sample - loss: 0.3937 - binary_accuracy: 0.8684
Epoch 3/200
126488/126488 [==============================] - 12s 93us/sample - loss: 0.3676 - binary_accuracy: 0.8835
Epoch 4/200
126488/126488 [==============================] - 12s 95us/sample - loss: 0.3468 - binary_accuracy: 0.8961
Epoch 5/200
126488/126488 [==============================] - 12s 95us/sample - loss: 0.3379 - binary_accuracy: 0.9011
Epoch 6/200
126488/126488 [==============================] - 12s 93us/sample - loss: 0.3261 - binary_accuracy: 0.9067
Epoch 7/200
126488/126488 [==============================] - 12s 93us/sample - loss: 0.3181 - binary_accuracy: 0.9116
Epoch 8/200
126488/126488 [==============================] - 12s 94us/sample - loss: 0.3106 - binary_accuracy: 0.9163
Epoch 9/200
126488/126488 [====

In [26]:
predictions = model.predict(x_testset, batch_size=None, verbose=1)

pred = np.zeros(predictions.shape[0]) 
pred[predictions[:,0]< predictions[:,1]] = 1 #merging predictions to 1d vector with value 0 or 1

72200/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [22]:
model.save('cnn.model80')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: cnn.model80/assets


In [0]:
#to aicrowd
im_pred = model.predict(im_test, batch_size=None, verbose=0)

val_pred = np.zeros(im_pred.shape[0]) 
val_pred[im_pred[:,0]< im_pred[:,1]] = 1

create_submission(pred, "c_as12uuubmission.csv")


In [0]:
#pour f1 score sur validation data

y_v = np.zeros(y_val.shape[0])
y_v[y_val[:,1] == 1] = 1

from sklearn.metrics import f1_score

f1 = f1_score(y_v, val_pred, average='binary')  
print('F1 score:', f1)

F1 score: 0.7447748012952605
